In [ ]:
# Make sure that caffe is on the python path:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd

from sklearn.decomposition import PCA as sklearnPCA
landMarks = pd.read_csv('../../processing/landmarking/mit/mitLandmarks.csv',index_col = 0)
imageList = list(set(landMarks.index.tolist()))
imageList.sort()
imList = [im[3:] for im in imageList]
print len(imList)
Dataset = 'mit'
MODEL = 'geometric'
homePath = '/home/lli-ms/'
allGeometric = np.loadtxt(homePath+'/features/'+MODEL+'_geometric_'+Dataset+\
           '_totalFeatures.csv', delimiter=',')
print allGeometric.shape

caffe_root = homePath+'caffe/'
pretrained_model_root = homePath+'caffe/'

# run this line one time only!
import sys
caffePython = pretrained_model_root + 'python'
if caffePython not in sys.path:
    sys.path.insert(0, caffePython)

import caffe
# Load mean
mu = np.load(caffe_root + 'python/caffe/imagenet/ilsvrc_2012_mean.npy')
mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
print 'mean-subtracted values:', zip('BGR', mu)

# Load the trained net
MODEL = 'caffeNet' #'caffeNet','vgg16','vggFace' or 'faceSNN'
    
MODEL_FILE = caffe_root + 'models/bvlc_reference_caffenet/deploy.prototxt'
PRETRAINED_FILE = caffe_root + 'models/bvlc_reference_caffenet/bvlc_reference_caffenet.caffemodel'

caffe.set_device(1)
caffe.set_mode_gpu()
if not os.path.isfile(PRETRAINED_FILE):
    print("No caffemodel!!!")
elif not os.path.isfile(MODEL_FILE):
    print("No MODEL !!!")
else:
    print "Defining the net!"
    net = caffe.Net(MODEL_FILE,
                PRETRAINED_FILE,
                caffe.TEST)
# input preprocessing: 'data' is the name of the input blob == net.inputs[0]
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
if MODEL != 'faceSNN':
    # subtract the dataset-mean value in each channel
    transformer.set_mean('data', mu)
for layer_name, param in net.params.iteritems():
    print layer_name + '\t' + str(param[0].data.shape), str(param[1].data.shape)
transformer.set_transpose('data', (2,0,1))
# the reference model operates on images in [0,255] range instead of [0,1]
transformer.set_raw_scale('data', 255) 
# the reference model has channels in BGR order instead of RGB
transformer.set_channel_swap('data', (2,1,0))
if MODEL == 'caffeNet':
    imgeReshape = [227,227]
    featureLayer = 'fc7'
if 'fc' in featureLayer:
    featureNum = net.params[featureLayer][1].data.shape[0]
else:
    featureNum = net.blobs[featureLayer].data.flatten().shape[0]/net.blobs[featureLayer].data.shape[0]
print featureNum
features = np.zeros([len(imList),featureNum])
totalNum = 0

# print len(imList)
for img in imList:
    imgName = os.path.basename(img)
    if imgName.endswith(('.jpg','.png')):
        input_image = caffe.io.load_image(img)
        net.blobs['data'].reshape(1,3,imgeReshape[0],imgeReshape[1])
        net.blobs['data'].data[...] = transformer.preprocess('data', input_image)
        out = net.forward()
        feat = net.blobs[featureLayer].data
        if Dataset =='twin':
            img_type = int(imgName[7:-4])/perImNum
            img_index = int(imgName[7:-4])%perImNum
            #print 'img_type:',img_type
            if img_type in img_type_num.keys():
                img_type_num[img_type] = img_type_num[img_type] + 1
                img_type_list[img_type][img_index] = img
            else:
                img_type_num[img_type] = 0
                img_type_list[img_type] = [None]*perImNum
                img_type_index[img_type] = type_index
                type_index +=1
            #print 'img_type_index:',img_type_index[img_type]
            features[img_type_index[img_type],img_type_num[img_type]] = feat.flatten()
        else:
            # need to be further revised!
            features[totalNum] = feat.flatten()
            #print features[totalNum]
        totalNum +=1
    else:
        print img
#print len(img_type_num)
print totalNum
caffe_Feature = features

2207
(2207, 4592)
mean-subtracted values: [('B', 104.0069879317889), ('G', 116.66876761696767), ('R', 122.6789143406786)]
Defining the net!
conv1	(96, 3, 11, 11) (96,)
conv2	(256, 48, 5, 5) (256,)
conv3	(384, 256, 3, 3) (384,)
conv4	(384, 192, 3, 3) (384,)
conv5	(256, 192, 3, 3) (256,)
fc6	(4096, 9216) (4096,)
fc7	(4096, 4096) (4096,)
fc8	(1000, 4096) (1000,)
4096
2207


In [ ]:
for j in range(1,50): 
    import random
    featureMat = caffe_Feature
    randomInd = range(featureMat.shape[0])
    random.shuffle(randomInd)
    print max(randomInd)
    featureMat = featureMat[randomInd,:]
    nSamples = featureMat.shape[0]
    testRatio = 0.2
    valiRatio = 0.2
    testFeatures = featureMat[:int(nSamples*testRatio),:]
    trainFeatures = featureMat[int(nSamples*testRatio):,:]
    valiFeatures = featureMat[-int(trainFeatures.shape[0]*valiRatio):,:]
    trainFeatures = trainFeatures[:-int(trainFeatures.shape[0]*valiRatio),:]
    print valiFeatures.shape, trainFeatures.shape, testFeatures.shape
    explained_variance = 700
    sklearn_pca = sklearnPCA(n_components=explained_variance, whiten  = True)
    trainfeature_transf = sklearn_pca.fit_transform(trainFeatures)
    print 'The number of PCs needed to retain %.3f variance is %d.' \
          % (explained_variance, trainfeature_transf.shape[1])
    print sklearn_pca.components_.shape
    print trainfeature_transf.shape
    testfeature_tansf = sklearn_pca.transform(testFeatures)
    print testfeature_tansf.shape
    valifeature_transf = sklearn_pca.transform(valiFeatures)

    radomImlist = np.asarray(imList)[randomInd]


    modelList = []
    optFeaNumList = []

    import sys
    import pandas as pd
    socialMeasures = '../Result/mit/socialMeasures.csv'
    socialMeasures = pd.read_csv(socialMeasures,index_col = 0)
    socialAttr = socialMeasures.columns.tolist()
    delElement = ['subage.1', 'submale.1', 'subrace.1','subage', 'submale',\
                  'subrace','catch', 'catchAns','catch.1','catchAns.1']
    social2Attr = [x for x in socialAttr if x not in delElement]

    PkgPath = homePath+'attractiveness_datamining/linjieCode/code'
    fName = "../Result/CaffeNet/regression_result_fc7_"+str(j)+".csv"
    text_file = open(fName, "w")
    d = 'Social Attributes, caffeNet_test, caffeNet_train, caffeNet_FeatureNum'+'\n'
    text_file.write(d)

    if PkgPath not in sys.path:
        sys.path.insert(0, PkgPath)
    #print sys.path
    from xVal_train_test import Train_Test

    import sklearn
    import numpy as np
    fullRating = np.zeros((2207,40))
    for attr in social2Attr:
        print attr
        #attr= social2Attr[9]，
        mean_rating = [socialMeasures.loc[imList[i][31:],attr] for i in range(len(imList))\
                   if imList[i][31:] in socialMeasures.index]
        mean_rating = map(float, mean_rating)
        mean_rating = np.array(mean_rating)
        radomRating = mean_rating[randomInd]
        testRating = radomRating[:int(nSamples*testRatio)]
        trainRating = radomRating[int(nSamples*testRatio):]
        valiRating = radomRating[-int(trainRating.shape[0]*valiRatio):]
        trainSubRating = trainRating[:-int(trainRating.shape[0]*valiRatio)]

        baseLine = mean_rating.mean()
        print 'mean rating: ', baseLine

        predictionModel = sklearn.linear_model.RidgeCV(alphas=np.logspace(-3,2,num=20), fit_intercept=True)
        myModel,optFeaNum,testC,trainC = Train_Test(trainSubRating,testRating,valiRating,trainfeature_transf, \
                                       testfeature_tansf,valifeature_transf,xVal = True,numTrain = 1,\
                                       pModel = predictionModel,getMaxMin = False,MODEL= MODEL, \
                                       plotPredActual = False,returnModel = True,returnValTrain = True)
        modelList.append(myModel)
        optFeaNumList.append(optFeaNum)
        
       
        d = attr+', '+ str(testC)+', '+str(trainC)+', '+str(optFeaNum)+'\n'
        text_file.write(d)
    text_file.close()

2206
(353, 4096) (1413, 4096) (441, 4096)
The number of PCs needed to retain 700.000 variance is 700.
(700, 4096)
(1413, 700)
(441, 700)
atypical
mean rating:  4.08446414454
Correlation:  0.318188987945
num of features:  162
Spearman Correlation:  0.29519829687
num of features:  172
R^2 score:  0.0906759948747
num of features:  81
MSE:  0.435127808225
num of features:  81
**************************Result of train and test**************************************
number of features: 162
On test set:
Residual sum of squares: 0.43
Variance score is: 0.05
Correlation between predicted ratings and actual ratings is: 0.2903
Spearman Correlation between predicted ratings and actual ratings is: 0.2979
 
On training set:
Residual sum of squares: 0.33
Variance score is: 0.28
Correlation between predicted ratings and actual ratings is: 0.5320
Spearman Correlation between predicted ratings and actual ratings is: 0.5136
**********************************************************************************

In [ ]:
result = np.zeros((40,3))
for j in range(1,50): 
    fName = "../Result/CaffeNet/regression_result_fc7_"+str(j)+".csv"
    df = pd.read_csv(fName,index_col = 0)
    result += df.as_matrix()
result = result/49
result_df = pd.DataFrame(result,              
                         index=df.index,
                         columns=df.columns)
result_df.to_csv('../Result/CaffeNet/combined_result_fc7.csv')
print result_df

In [ ]:
totalRating = np.concatenate([trainSubRating,valiRating,testRating])
print totalRating.shape
totalFeature = np.concatenate([trainfeature_transf_vgg[:,:optFeaNum_vgg],valifeature_transf_vgg[:,:optFeaNum_vgg],\
                               testfeature_tansf_vgg[:,:optFeaNum_vgg]])
print totalFeature.shape
fullRating[:,social2Attr.index(attr)] = myModel_vgg.predict(totalFeature)